In [ ]:
!pip install transformers torch sentence-transformers faiss-cpu accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 960.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 67.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from huggingface_hub import login
login(token="your_hf_token")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"  # Change to a smaller model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_csv("/content/books.csv")  # Replace with actual filename
print(df.head())


          isbn13      isbn10           title subtitle  \
0  9780002005883  0002005883          Gilead      NaN   
1  9780002261982  0002261987    Spider's Web  A Novel   
2  9780006163831  0006163831    The One Tree      NaN   
3  9780006178736  0006178731  Rage of angels      NaN   
4  9780006280897  0006280897  The Four Loves      NaN   

                           authors                     categories  \
0               Marilynne Robinson                        Fiction   
1  Charles Osborne;Agatha Christie  Detective and mystery stories   
2             Stephen R. Donaldson               American fiction   
3                   Sidney Sheldon                        Fiction   
4              Clive Staples Lewis                 Christian life   

                                           thumbnail  \
0  http://books.google.com/books/content?id=KQZCP...   
1  http://books.google.com/books/content?id=gA5GP...   
2  http://books.google.com/books/content?id=OmQaw...   
3  http://books.go

In [ ]:
df = df[['title', 'authors', 'categories', 'description', 'average_rating']]
df.dropna(inplace=True)  # Remove rows with missing values
print(df.head())  # Check cleaned data


            title                          authors  \
0          Gilead               Marilynne Robinson   
1    Spider's Web  Charles Osborne;Agatha Christie   
2    The One Tree             Stephen R. Donaldson   
3  Rage of angels                   Sidney Sheldon   
4  The Four Loves              Clive Staples Lewis   

                      categories  \
0                        Fiction   
1  Detective and mystery stories   
2               American fiction   
3                        Fiction   
4                 Christian life   

                                         description  average_rating  
0  A NOVEL THAT READERS and critics have been eag...            3.85  
1  A new 'Christie for Christmas' -- a full-lengt...            3.83  
2  Volume Two of Stephen Donaldson's acclaimed se...            3.97  
3  A memorable, mesmerizing heroine Jennifer -- b...            3.93  
4  Lewis' work on the nature of love divides love...            4.15  


<ipython-input-5-661d4db53607>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)  # Remove rows with missing values


In [ ]:
from sentence_transformers import SentenceTransformer

# Load the embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert book descriptions into embeddings
book_embeddings = embedding_model.encode(df["description"].tolist())

print("Embeddings shape:", book_embeddings.shape)  # Check the output size


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings shape: (6412, 384)


In [ ]:
import faiss
import numpy as np

# Create FAISS index
dimension = book_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to FAISS index
index.add(np.array(book_embeddings))

# Store book titles for retrieval
book_titles = df["title"].tolist()

print("FAISS index created with", index.ntotal, "books.")


FAISS index created with 6412 books.


In [ ]:
def recommend_books(user_query, category=None, author=None, top_n=5, min_rating=3.5):
    user_embedding = embedding_model.encode([user_query])
    _, similar_books = index.search(np.array(user_embedding), top_n * 10)

    recommendations = []
    for i in similar_books[0]:
        book_title = df.iloc[i]["title"]
        book_author = df.iloc[i]["authors"]
        book_category = df.iloc[i]["categories"]
        book_rating = df.iloc[i]["average_rating"]
        book_thumbnail = "https://via.placeholder.com/80x120"  # Default placeholder

        if ((not category or category.lower() in book_category.lower()) and
            (not author or author.lower() in book_author.lower()) and
            book_rating >= min_rating):
            recommendations.append((book_title, book_author, book_category, book_rating, book_thumbnail))

        if len(recommendations) == top_n:
            break

    recommendations.sort(key=lambda x: x[3], reverse=True)

    html = ""
    for book in recommendations:
        html += f"""
        <div style="display: flex; align-items: center; margin-bottom: 15px;">
            <img src="{book[4]}" style="width: 80px; height: 120px; margin-right: 15px; border-radius: 5px;">
            <div>
                <strong>📖 {book[0]}</strong><br>
                👤 <em>{book[1]}</em><br>
                📂 {book[2]}<br>
                ⭐ {book[3]}
            </div>
        </div>
        """

    display.display(HTML(html))
    return recommendations

# Test the function
user_query = "A thrilling mystery novel"
recommended_books = recommend_books(user_query, category="Mystery", min_rating=3.5)


In [ ]:
print(df.columns)


Index(['title', 'authors', 'categories', 'description', 'average_rating'], dtype='object')


In [ ]:
# Ensure FAISS index count matches DataFrame row count
print(f"FAISS Index Size: {index.ntotal}, DataFrame Rows: {df.shape[0]}")


FAISS Index Size: 6412, DataFrame Rows: 6412


In [ ]:
# Example: Encode user input query
user_query = "Detective mystery novel"
query_embedding = embedding_model.encode(user_query)


In [ ]:
D, I = index.search(np.array([query_embedding]), k=5)  # Retrieve top 5 matches
print("FAISS Indices:", I)


FAISS Indices: [[3778 4045 2368 2118 2298]]


In [ ]:
recommended_books = df.iloc[I[0]][["title", "authors", "categories", "average_rating"]]

for _, book in recommended_books.iterrows():
    print(f"📖 {book['title']}\n👤 {book['authors']}\n📂 {book['categories']}\n⭐ {book['average_rating']}\n")


📖 The Murders in the Rue Morgue
👤 Edgar Allan Poe
📂 Fiction
⭐ 3.87

📖 Medicus and the Disappearing Dancing Girls
👤 Ruth Downie
📂 Detective and mystery stories
⭐ 3.73

📖 The Simple Art of Murder
👤 Raymond Chandler
📂 Fiction
⭐ 4.16

📖 Neon Lit:city of Glass
👤 Bob Callahan
📂 Fiction
⭐ 3.92

📖 Filth
👤 Irvine Welsh
📂 Fiction
⭐ 3.74



In [ ]:
import numpy as np
import faiss
import pickle

# Function to get refined recommendations
def get_recommendations(query, top_k=10, min_rating=3.5):
    # Generate embedding for the query
    query_embedding = embedding_model.encode(query)

    # Search FAISS index
    D, I = index.search(np.array([query_embedding]), k=top_k)

    # Retrieve books
    recommended_books = df.iloc[I[0]]

    # 1️⃣ Filter by category (optional: adjust to match query genre)
    if "categories" in df.columns:
        category = recommended_books["categories"].value_counts().idxmax()  # Most common category
        recommended_books = recommended_books[recommended_books["categories"] == category]

    # 2️⃣ Boost higher-rated books
    recommended_books = recommended_books[recommended_books["average_rating"] >= min_rating]

    # 3️⃣ Ensure diversity by dropping duplicates
    recommended_books = recommended_books.drop_duplicates(subset=["title"])

    # Show recommendations
    for _, book in recommended_books.iterrows():
        print(f"📖 {book['title']}\n👤 {book['authors']}\n📂 {book['categories']}\n⭐ {book['average_rating']}\n")

# Example usage
get_recommendations("Detective mystery novel", top_k=10, min_rating=4.0)


📖 The Simple Art of Murder
👤 Raymond Chandler
📂 Fiction
⭐ 4.16

📖 The Long Goodbye
👤 Raymond Chandler
📂 Fiction
⭐ 4.22



In [ ]:
import pickle
import faiss

# Save embeddings
with open("embeddings.pkl", "wb") as f:
    pickle.dump(book_embeddings, f)

# Save FAISS index
faiss.write_index(index, "book_recommender.faiss")

# Save book titles
with open("book_titles.pkl", "wb") as f:
    pickle.dump(book_titles, f)

print("Embeddings, FAISS index, and book titles saved successfully!")


Embeddings, FAISS index, and book titles saved successfully!


In [ ]:
import pickle
import faiss

# Save embeddings
with open("embeddings.pkl", "wb") as f:
    pickle.dump(book_embeddings, f)

# Save FAISS index
faiss.write_index(index, "book_recommender.faiss")

# Save book titles
with open("book_titles.pkl", "wb") as f:
    pickle.dump(book_titles, f)

print("Embeddings, FAISS index, and book titles saved successfully!")


Embeddings, FAISS index, and book titles saved successfully!


In [ ]:
print("Embeddings Shape:", book_embeddings.shape)
print("First Embedding:", book_embeddings[0] if len(book_embeddings) > 0 else "No embeddings found")


Embeddings Shape: (6412, 384)
First Embedding: [-4.01278846e-02 -2.79154233e-03 -7.63532519e-03 -5.30540273e-02
 -4.97787893e-02  3.09776533e-02  1.41330948e-02  2.76363511e-02
  8.58496949e-02  2.51838826e-02 -4.44324082e-03  1.79786161e-02
  7.72379711e-02 -5.32127693e-02 -6.34565502e-02  6.11929782e-03
  1.08863162e-02  1.14107579e-02 -3.36236656e-02  5.21181114e-02
  3.41706425e-02  7.66324718e-03 -3.10708256e-03 -5.14379479e-02
 -1.89642105e-02  3.22419815e-02 -1.24223093e-02  2.44239420e-02
 -7.79712647e-02 -6.10903557e-03  3.15680280e-02 -2.42168475e-02
  1.01487681e-01 -3.10994666e-02 -2.87968330e-02  2.07797494e-02
  5.11247255e-02  9.70200151e-02  2.82009561e-02 -9.05424182e-04
  7.92390257e-02  4.09311987e-02  1.46233616e-03 -2.97985133e-02
  7.84023572e-03 -4.20811623e-02 -3.25165642e-03 -6.72552213e-02
 -2.29847915e-02  3.24172503e-03 -2.09441110e-02 -8.05331692e-02
  1.65120121e-02 -1.07175242e-02  3.80684808e-02  1.18820891e-01
  2.41492596e-02 -2.99296714e-02 -2.0452316

In [ ]:
# Load embeddings
with open("embeddings.pkl", "rb") as f:
    book_embeddings = pickle.load(f)

# Load FAISS index
index = faiss.read_index("book_recommender.faiss")

# Load book titles
with open("book_titles.pkl", "rb") as f:
    book_titles = pickle.load(f)

print("Loaded FAISS index with", index.ntotal, "books.")


Loaded FAISS index with 0 books.


In [ ]:
import faiss

# Load the correct FAISS index file
index = faiss.read_index("book_recommender.faiss")

print("FAISS index loaded successfully!")


FAISS index loaded successfully!


In [ ]:
import faiss
import numpy as np

# Load the FAISS index
index = faiss.read_index("book_recommender.faiss")
print("FAISS index loaded successfully!")

# Create a sample query vector (Modify it based on your dataset)
# Assuming your vectors are of dimension `d`, replace `d` with the actual dimension
d = index.d  # Get dimension of vectors in the index
query_vector = np.random.rand(d).astype('float32')  # Replace with actual book vector

# Number of nearest neighbors to retrieve
k = 5  # Get top 5 recommendations

# Perform similarity search
distances, indices = index.search(query_vector.reshape(1, -1), k)

# Print the results
print("Recommended Books (Indices):", indices[0])
print("Distances:", distances[0])


FAISS index loaded successfully!
Recommended Books (Indices): [-1 -1 -1 -1 -1]
Distances: [3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]


In [ ]:
print("Number of vectors in the index:", index.ntotal)


Number of vectors in the index: 0


In [ ]:
# Load FAISS index
index = faiss.read_index("book_recommender.faiss")

# Load book titles
with open("book_titles.pkl", "rb") as f:
    book_titles = pickle.load(f)

# Load book embeddings
with open("embeddings.pkl", "rb") as f:
    book_embeddings = pickle.load(f)

# Check index size
print("Number of vectors in the index:", index.ntotal)


Number of vectors in the index: 0


In [ ]:
import faiss
import pickle
import numpy as np

# Load book titles
with open("book_titles.pkl", "rb") as f:
    book_titles = pickle.load(f)

# Load book embeddings
with open("embeddings.pkl", "rb") as f:
    book_embeddings = pickle.load(f)

# Convert embeddings to NumPy array
book_embeddings = np.array(book_embeddings).astype(np.float32)

# Check embeddings shape
print("Embeddings shape:", book_embeddings.shape)


Embeddings shape: (6412, 384)


In [ ]:
import faiss
import numpy as np

# Define FAISS index based on embedding dimension (384)
embedding_dim = 384
index = faiss.IndexFlatL2(embedding_dim)

# Convert embeddings to NumPy array (if not already)
book_embeddings = np.array(book_embeddings).astype(np.float32)

# Add embeddings to FAISS index
index.add(book_embeddings)

# Save the updated FAISS index
faiss.write_index(index, "book_recommender.faiss")

print("✅ FAISS index rebuilt and saved successfully!")
print("Number of vectors in the FAISS index:", index.ntotal)


✅ FAISS index rebuilt and saved successfully!
Number of vectors in the FAISS index: 6412


In [ ]:
import numpy as np

def recommend_books(query_embedding, index, book_titles, k=5):
    """
    Recommend books based on the query embedding.
    """
    query_embedding = np.array(query_embedding).astype(np.float32).reshape(1, -1)

    # Search for top-k nearest neighbors
    distances, indices = index.search(query_embedding, k)

    # Print recommended book titles
    print("\n📚 Recommended Books:")
    for i, idx in enumerate(indices[0]):
        if idx != -1:  # Valid index check
            print(f"{i+1}. {book_titles[idx]} (Distance: {distances[0][i]:.4f})")
        else:
            print(f"{i+1}. No match found")

    return indices[0]

# Example: Use a random book embedding as a query
query_embedding = book_embeddings[0]  # Using the first book as an example
recommend_books(query_embedding, index, book_titles)



📚 Recommended Books:
1. Gilead (Distance: 0.0000)
2. Mr. Ives' Christmas (Distance: 0.9949)
3. The Epic of Gilgamesh (Distance: 1.0008)
4. The Complete Stories of Evelyn Waugh (Distance: 1.0210)
5. The House of the Scorpion (Distance: 1.0448)


array([   0,  337, 4754, 1625, 3990])

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the FAISS index
index = faiss.read_index("book_recommender.faiss")

# Load book titles
with open("book_titles.pkl", "rb") as f:
    book_titles = pickle.load(f)

# Load the same model used to generate embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")  # Ensure this matches the original model

# 🔹 Step 1: Get user input for a book description
new_book_description = "A thrilling mystery novel about a detective solving a decades-old case."

# 🔹 Step 2: Convert the description into an embedding
new_book_embedding = model.encode(new_book_description)
new_book_embedding = np.array([new_book_embedding], dtype=np.float32)  # Reshape for FAISS

# 🔹 Step 3: Search FAISS for similar books
k = 5  # Number of recommendations
distances, indices = index.search(new_book_embedding, k)

# 🔹 Step 4: Display results
print("\n📚 Recommended Books:")
for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
    if idx != -1:  # Ensure a valid book index
        print(f"{i+1}. {book_titles[idx]} (Distance: {dist:.4f})")
    else:
        print(f"{i+1}. No match found.")




📚 Recommended Books:
1. The Murders in the Rue Morgue (Distance: 0.5557)
2. The Long Goodbye (Distance: 0.7801)
3. Neon Lit:city of Glass (Distance: 0.8200)
4. Filth (Distance: 0.8380)
5. The Simple Art of Murder (Distance: 0.8769)


In [ ]:
import pickle

with open("book_titles.pkl", "rb") as f:
    book_data = pickle.load(f)

print(book_data[:5])  # Print the first 5 records


['Gilead', "Spider's Web", 'The One Tree', 'Rage of angels', 'The Four Loves']


In [ ]:
import pandas as pd
# Load CSV (example)
df = pd.read_csv("/content/books.csv")  # Ensure it has 'title' & 'thumbnail'

# Convert to a list of dictionaries
book_data = df[["title", "thumbnail"]].to_dict(orient="records")

# Save to `book_titles.pkl`
with open("book_titles_thumbnail.pkl", "wb") as f:
    pickle.dump(book_data, f)

print("✅ `book_titles_thumbnail.pkl` saved in correct format!")


✅ `book_titles_thumbnail.pkl` saved in correct format!


In [ ]:
with open("book_titles_thumbnail.pkl", "rb") as f:
    book_data = pickle.load(f)

print(book_data[:5])  # Print the first 5 records

[{'title': 'Gilead', 'thumbnail': 'http://books.google.com/books/content?id=KQZCPgAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api'}, {'title': "Spider's Web", 'thumbnail': 'http://books.google.com/books/content?id=gA5GPgAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api'}, {'title': 'The One Tree', 'thumbnail': 'http://books.google.com/books/content?id=OmQawwEACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api'}, {'title': 'Rage of angels', 'thumbnail': 'http://books.google.com/books/content?id=FKo2TgANz74C&printsec=frontcover&img=1&zoom=1&source=gbs_api'}, {'title': 'The Four Loves', 'thumbnail': 'http://books.google.com/books/content?id=XhQ5XsFcpGIC&printsec=frontcover&img=1&zoom=1&source=gbs_api'}]


In [ ]:
df.head(5)

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [ ]:
import pickle
import pandas as pd

# Load the existing book data with titles and thumbnails
with open("book_titles_thumbnail.pkl", "rb") as f:
    book_data = pickle.load(f)

# Load the original dataset (modify the filename if different)
dataset = pd.read_csv("/content/books.csv")  # Ensure this CSV contains all book details

# Convert dataset to a dictionary for faster lookup
dataset_dict = {row["title"]: row for _, row in dataset.iterrows()}

# Update book_data with additional information
for book in book_data:
    title = book["title"]

    if title in dataset_dict:
        book.update({
            "authors": dataset_dict[title]["authors"].split(", ") if pd.notna(dataset_dict[title]["authors"]) else [],
            "categories": dataset_dict[title]["categories"].split(", ") if pd.notna(dataset_dict[title]["categories"]) else [],
            "description": dataset_dict[title]["description"] if pd.notna(dataset_dict[title]["description"]) else "No description available.",
            "average_rating": dataset_dict[title]["average_rating"] if pd.notna(dataset_dict[title]["average_rating"]) else "N/A",
            "ratings_count": int(dataset_dict[title]["ratings_count"]) if pd.notna(dataset_dict[title]["ratings_count"]) else 0
        })
    else:
        print(f"Warning: No data found for '{title}' in the dataset.")

# Save the updated file
with open("book_titles_thumbnail.pkl", "wb") as f:
    pickle.dump(book_data, f)

print("✅ File updated successfully!")


✅ File updated successfully!


In [ ]:
with open("book_titles_thumbnail.pkl", "rb") as f:
    book_data = pickle.load(f)

print(book_data[:5])  # Print the first 5 records

[{'title': 'Gilead', 'author': 'Unknown Author', 'category': 'Unknown Category', 'rating': 0.0, 'thumbnail': 'http://books.google.com/books/content?id=KQZCPgAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api', 'authors': ['Marilynne Robinson'], 'categories': ['Fiction'], 'description': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton,